# **Лабораторная работа №3**

### **Задание 1**.

Классификация

In [1]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

df = pd.read_csv("../data/classification/csgo_task_update.csv", index_col=0).drop_duplicates()
display(df)

y = df['bomb_planted']
X = df.drop(['bomb_planted'], axis=1)

underSampler = RandomUnderSampler(sampling_strategy='majority')
X_under_sample, y_under_sample = underSampler.fit_resample(X, y)

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_under_sample, y_under_sample, test_size=0.2, random_state = 4)

scaler = StandardScaler()

scaler.fit(X_train_clf, y_train_clf)
X_train_std_clf = scaler.transform(X_train_clf)
X_test_std_clf = scaler.transform(X_test_clf)

,time_left,ct_score,t_score,bomb_planted,ct_health,t_health,ct_money,t_money,ct_players_alive,t_players_alive,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
0,175.00,0.0,0.0,0,500.0,500.0,4000.0,4000.0,5.0,5.0,0,1,0,0,0,0,0,0
1,156.03,0.0,0.0,0,500.0,500.0,600.0,650.0,5.0,5.0,0,1,0,0,0,0,0,0
2,96.03,0.0,0.0,0,391.0,400.0,750.0,500.0,4.0,4.0,0,1,0,0,0,0,0,0
3,76.03,0.0,0.0,0,391.0,400.0,750.0,500.0,4.0,4.0,0,1,0,0,0,0,0,0
4,174.97,1.0,0.0,0,500.0,500.0,18350.0,10750.0,5.0,5.0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,1,200.0,242.0,100.0,5950.0,2.0,4.0,0,0,0,0,0,0,1,0
122406,174.93,11.0,15.0,0,500.0,500.0,11500.0,23900.0,5.0,5.0,0,0,0,0,0,0,1,0
122407,114.93,11.0,15.0,0,500.0,500.0,1200.0,6700.0,5.0,5.0,0,0,0,0,0,0,1,0
122408,94.93,11.0,15.0,0,500.0,500.0,1200.0,6700.0,5.0,5.0,0,0,0,0,0,0,1,0


Поиск наилучших гиперпараметров

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3]
}

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)

clf = DecisionTreeClassifier(criterion= best_params['criterion'],
                             splitter= best_params['splitter'],
                             max_depth= best_params['max_depth'],
                             min_samples_split= best_params['min_samples_split'],
                             min_samples_leaf= best_params['min_samples_leaf']
                             )
clf.fit(X_train_std_clf, y_train_clf)
y_pred_clf = clf.predict(X_test_std_clf)
print(clf.score(X_test_std_clf, y_test_clf))

{'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 3, 'splitter': 'best'}
0.9771647789550603


Метрики

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.9769820971867008
Confusion matrix:
 [[2689   95]
 [  31 2659]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2784
           1       0.97      0.99      0.98      2690

    accuracy                           0.98      5474
   macro avg       0.98      0.98      0.98      5474
weighted avg       0.98      0.98      0.98      5474



In [ ]:
from sklearn import tree

text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_0 <= -0.43
|   |--- feature_3 <= 0.06
|   |   |--- feature_0 <= -0.53
|   |   |   |--- feature_0 <= -0.90
|   |   |   |   |--- feature_0 <= -0.90
|   |   |   |   |   |--- feature_3 <= -0.72
|   |   |   |   |   |   |--- feature_0 <= -0.90
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_0 >  -0.90
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- feature_3 >  -0.72
|   |   |   |   |   |   |--- feature_4 <= -0.46
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_4 >  -0.46
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_0 >  -0.90
|   |   |   |   |   |--- feature_3 <= -1.11
|   |   |   |   |   |   |--- feature_0 <= -0.90
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_0 >  -0.90
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_3 >  -1.11
|   |   |   |   |   |   |--- feature_4 <= -0.32
|   |   |   |   |   |   |   |--- clas

Регрессия

In [ ]:
from sklearn.tree import DecisionTreeRegressor

df = pd.read_csv("/content/drive/MyDrive/ML/RegressionFirstLab/trip_duration_update.csv", index_col=0).head(50000)
display(df)

y = df["trip_duration"]
X = df.drop(["trip_duration"], axis=1)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
scaler.fit(X_train_reg, y_train_reg)
X_train_reg = scaler.transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,-73.953918,40.778873,-73.963875,40.771164,400
1,-73.988312,40.731743,-73.994751,40.694931,1100
2,-73.997314,40.721458,-73.948029,40.774918,1635
3,-73.961670,40.759720,-73.956779,40.780628,1141
4,-74.017120,40.708469,-73.988182,40.740631,848
...,...,...,...,...,...
49995,-73.963600,40.765484,-73.974869,40.751869,406
49996,-73.981430,40.778862,-73.973000,40.790333,126
49997,-73.995628,40.721668,-73.981697,40.724522,387
49998,-73.968208,40.765339,-73.933357,40.764420,747


Поиск наилучших гиперпараметров

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


param_grid = {
    'criterion': ['squared_error', 'absolute_error'],
    'splitter': ['best'],
    'max_depth': [3, 4, 5]
}

grid_search = RandomizedSearchCV(DecisionTreeRegressor(), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)

reg = DecisionTreeRegressor(criterion= best_params['criterion'],
                             splitter= best_params['splitter'],
                             max_depth= best_params['max_depth'])
reg.fit(X_train_reg, y_train_reg)
y_pred_reg = reg.predict(X_test_reg)
print(reg.score(X_test_reg, y_test_reg))

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'splitter': 'best', 'max_depth': 5, 'criterion': 'absolute_error'}
0.0240458234271248


In [ ]:


print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 430.3991
MSE: 6993937.6557
RMSE: 2644.605387520036
MAPE: 0.9440858559589781
R^2: 0.0240458234271248


In [ ]:
from sklearn import tree

text_representation = tree.export_text(reg)
print(text_representation)

|--- feature_0 <= 2.31
|   |--- feature_2 <= 1.59
|   |   |--- feature_3 <= -1.56
|   |   |   |--- feature_1 <= -0.64
|   |   |   |   |--- feature_1 <= -1.59
|   |   |   |   |   |--- value: [502.00]
|   |   |   |   |--- feature_1 >  -1.59
|   |   |   |   |   |--- value: [1092.50]
|   |   |   |--- feature_1 >  -0.64
|   |   |   |   |--- feature_1 <= -0.05
|   |   |   |   |   |--- value: [1556.50]
|   |   |   |   |--- feature_1 >  -0.05
|   |   |   |   |   |--- value: [1954.00]
|   |   |--- feature_3 >  -1.56
|   |   |   |--- feature_3 <= -0.83
|   |   |   |   |--- feature_1 <= -0.37
|   |   |   |   |   |--- value: [547.00]
|   |   |   |   |--- feature_1 >  -0.37
|   |   |   |   |   |--- value: [1134.00]
|   |   |   |--- feature_3 >  -0.83
|   |   |   |   |--- feature_1 <= -0.99
|   |   |   |   |   |--- value: [970.50]
|   |   |   |   |--- feature_1 >  -0.99
|   |   |   |   |   |--- value: [571.00]
|   |--- feature_2 >  1.59
|   |   |--- feature_2 <= 3.62
|   |   |   |--- feature_0 <= 0.

### **Задание 2**


### BaggingClassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier

base_estimator = DecisionTreeClassifier(random_state=0)

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0]
}

grid_search = GridSearchCV(BaggingClassifier(estimator=base_estimator,
                                random_state=0), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)

y_pred_clf = grid_search.predict(X_test_std_clf)
print(grid_search.score(X_test_std_clf, y_test_clf))

{'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 30}
0.979539641943734


In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.979539641943734
Confusion matrix:
 [[2696   88]
 [  24 2666]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2784
           1       0.97      0.99      0.98      2690

    accuracy                           0.98      5474
   macro avg       0.98      0.98      0.98      5474
weighted avg       0.98      0.98      0.98      5474



### BaggingRegressor

In [ ]:
from sklearn.ensemble import BaggingRegressor

base_estimator = DecisionTreeRegressor(random_state=0)

param_grid = {
    'n_estimators': [10, 20, 30],
    'max_samples': [0.5, 1.0],
    'max_features': [0.5, 1.0]
}

grid_search = GridSearchCV(BaggingRegressor(estimator=base_estimator,
                               random_state=0), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)

y_pred_reg = grid_search.predict(X_test_reg)
print(grid_search.score(X_test_reg, y_test_reg))

{'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 20}
-4.796791044573684


In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 512.80666
MSE: 41541289.685582995
RMSE: 6445.253267760934
MAPE: 0.8969569907173697
R^2: -4.796791044573684


### GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    'n_estimators': [100, 200, 300]
}

grid_search = GridSearchCV(GradientBoostingClassifier(random_state=0), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)

y_pred_clf = grid_search.predict(X_test_std_clf)
print(grid_search.score(X_test_std_clf, y_test_clf))

{'n_estimators': 200}
0.9815491413956887


In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.9815491413956887
Confusion matrix:
 [[2705   79]
 [  22 2668]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2784
           1       0.97      0.99      0.98      2690

    accuracy                           0.98      5474
   macro avg       0.98      0.98      0.98      5474
weighted avg       0.98      0.98      0.98      5474



### GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.0001]
}

grid_search = RandomizedSearchCV(GradientBoostingRegressor(random_state=0), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)

y_pred_reg = grid_search.predict(X_test_reg)
print(grid_search.score(X_test_reg, y_test_reg))

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 300, 'learning_rate': 0.0001}
0.0002081307030576962


In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 606.1651187060999
MSE: 7164764.668658031
RMSE: 2676.707804123945
MAPE: 1.2833286541424773
R^2: 0.0002081307030576962


### StackingClassifier

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

base_estimators = [('svc', SVC(probability=True)), ('dt', DecisionTreeClassifier())]

final_estimator = LogisticRegression()

clf = StackingClassifier(estimators=base_estimators, final_estimator=final_estimator)
clf.fit(X_train_std_clf, y_train_clf)
y_pred_clf = clf.predict(X_test_std_clf)
print(clf.score(X_test_std_clf, y_test_clf))                                

0.9649251004749726


In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.9649251004749726
Confusion matrix:
 [[2695   89]
 [ 103 2587]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      2784
           1       0.97      0.96      0.96      2690

    accuracy                           0.96      5474
   macro avg       0.96      0.96      0.96      5474
weighted avg       0.96      0.96      0.96      5474



### StackingRegressor

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

base_estimators = [('svr', SVR()), ('dt', DecisionTreeRegressor())]

final_estimator = LinearRegression()

reg = StackingRegressor(estimators=base_estimators, final_estimator=final_estimator)
reg.fit(X_train_reg, y_train_reg)

y_pred_reg = reg.predict(X_test_reg)
print(reg.score(X_test_reg, y_test_reg))

0.022495847337780095


In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 456.8821483214351
MSE: 7005045.181439343
RMSE: 2646.7045890010736
MAPE: 1.0244852871332037
R^2: 0.022495847337780095


### Задание 3. 
### Попробовать для решения задачи регрессии и классификации одну из библиотек (на выбор)

### LightGBM

### Классификация

In [ ]:
from lightgbm import LGBMClassifier


param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}


grid_search = GridSearchCV(LGBMClassifier(), param_grid)
grid_search.fit(X_train_std_clf, y_train_clf)
best_params = grid_search.best_params_
print(best_params)


y_pred_clf = grid_search.predict(X_test_std_clf)
print(grid_search.score(X_test_std_clf, y_test_clf))

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
0.9786262331019364


In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.9786262331019364
Confusion matrix:
 [[2695   89]
 [  28 2662]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2784
           1       0.97      0.99      0.98      2690

    accuracy                           0.98      5474
   macro avg       0.98      0.98      0.98      5474
weighted avg       0.98      0.98      0.98      5474



### Регрессия

In [ ]:
from lightgbm import LGBMRegressor


param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}


grid_search = GridSearchCV(LGBMRegressor(), param_grid)
grid_search.fit(X_train_reg, y_train_reg)
best_params = grid_search.best_params_
print(best_params)


y_pred_reg = grid_search.predict(X_test_reg)
print(grid_search.score(X_test_reg, y_test_reg))

{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
-0.26990818494513635


In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 541.5469259371781
MSE: 9100487.386772538
RMSE: 3016.7014082889505
MAPE: 1.1805900256682418
R^2: -0.26990818494513635


### Задание 4.
### Реализовать обучения и вычисления дерева с использованием алгоритма CART для задачи регрессии и классификации.

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

In [ ]:
class CART():
    def __init__(self, min_samples_split=2, max_depth=2, isRegression = False):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.isRegression = isRegression
        
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
    
        if self.mode == "entropy":
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        else:
            gain = self.mean_squared_error(parent) - (weight_l*self.mean_squared_error(l_child) + weight_r*self.mean_squared_error(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
        
    def mean_squared_error(self, y):
        mean = np.mean(y)
        mse = np.mean((y - mean)**2)
        return mse
        
    def calculate_leaf_value(self, Y):
        if self.isRegression:
            return np.mean(Y)
        Y = list(Y)
        return max(Y, key=Y.count)
       
    def fit(self, X, Y):
        if self.isRegression:
            self.mode = "mse"
        else:
            self.mode = "entropy"
        dataset = np.column_stack((X, Y))
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

### Классификация

In [ ]:
tree = CART()
print(y_train_clf.shape)
print(X_train_std_clf.shape)
tree.fit(X_train_std_clf, y_train_clf)
y_pred_clf = tree.predict(X_test_std_clf)

(21894,)
(21894, 17)


In [ ]:
print(f"Accuracy: {accuracy_score(y_test_clf, y_pred_clf)}")
print(f"Confusion matrix:\n {confusion_matrix(y_test_clf, y_pred_clf)}")
print(f"Precision, Recall, F-score:\n{classification_report(y_test_clf, y_pred_clf)}")

Accuracy: 0.9413591523565948
Confusion matrix:
 [[2564  220]
 [ 101 2589]]
Precision, Recall, F-score:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      2784
           1       0.92      0.96      0.94      2690

    accuracy                           0.94      5474
   macro avg       0.94      0.94      0.94      5474
weighted avg       0.94      0.94      0.94      5474



### Регрессия

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ML/RegressionFirstLab/trip_duration_update.csv", index_col=0).head(10000)
display(df)

y = df['trip_duration'].values
X = df.drop(['trip_duration'], axis=1).values

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2)


treeReg = CART(isRegression=True)
treeReg.fit(X_train_reg, y_train_reg)
y_pred_reg = treeReg.predict(X_test_reg)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,-73.953918,40.778873,-73.963875,40.771164,400
1,-73.988312,40.731743,-73.994751,40.694931,1100
2,-73.997314,40.721458,-73.948029,40.774918,1635
3,-73.961670,40.759720,-73.956779,40.780628,1141
4,-74.017120,40.708469,-73.988182,40.740631,848
...,...,...,...,...,...
9995,-73.971199,40.788700,-74.015770,40.707981,2090
9996,-73.966499,40.753269,-73.965927,40.764671,212
9997,-73.980530,40.738190,-73.994179,40.740410,379
9998,-73.975517,40.749168,-73.975258,40.787720,925


In [ ]:
print(f'MAE: {mean_absolute_error(y_test_reg, y_pred_reg)}')
print(f'MSE: {mean_squared_error(y_test_reg, y_pred_reg)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_reg, y_pred_reg))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test_reg, y_pred_reg))}')
print(f'R^2: {r2_score(y_test_reg, y_pred_reg)}')

MAE: 426.3746066545871
MSE: 291926.9721446511
RMSE: 540.3026671641101
MAPE: 1.0789668641631387
R^2: 0.211177782301059


### BaggingRealization